In [1]:
import sys
import pandas as pd
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('expand_frame_repr', False)
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import os
import datetime as dt
import sklearn
import re

In [77]:
if os.getcwd()[9:14] == 'ssoma':
    file_path = "C:\\Users\\ssoma\\OneDrive - Bayer\\Personal Data-R90PQ30R\\Analytics\\house-pricing\\input-data\\st-charles\\"
else:
    file_path = "C:\\Users\\sivac\\Documents\\Analytics\\house-pricing\\input-data\\st-charles\\"


file_list = os.listdir(file_path)
for i in file_list:
    i = file_path + i
    file = pd.read_csv(i)
    
    if 'df' in globals():
        df = pd.concat([df, file])
    else:
        df = file

print(f'Raw dataset shape: {df.shape}')

df_new = df[['MAILING_ADDR', 'MAILING_ZIP', 'SUBDIVISION', 'SCHOOL_DISTRICT', 'LOT_SIZE', 'YEAR_BUILT', 'QUALITY_CODE', 'BEDROOMS', 'BATHROOMS',
            'HALF_BATHROOMS', 'TOTAL_ROOMS', 'ARCHITECTURAL_STYLE', 'TOTAL_AREA', 'BASE_AREA', 'BASEMENT_AREA','BASEMENT_FINISHED_AREA',
            'PARKING_AREA', 'LAND_VALUE', 'LAST_SALE_PRICE', 'LAST_SALE_DATE', 'MARKET_VALUE', 'LAST_SALE_UPSF']]

df_new = df_new.drop_duplicates().reset_index(drop=True)
print(f'Raw dataset shape with duplicates removed: {df_new.shape}')
df_new.head()

Raw dataset shape: (5424, 36)
Raw dataset shape with duplicates removed: (5172, 22)


,MAILING_ADDR,MAILING_ZIP,SUBDIVISION,SCHOOL_DISTRICT,LOT_SIZE,YEAR_BUILT,QUALITY_CODE,BEDROOMS,BATHROOMS,HALF_BATHROOMS,TOTAL_ROOMS,ARCHITECTURAL_STYLE,TOTAL_AREA,BASE_AREA,BASEMENT_AREA,BASEMENT_FINISHED_AREA,PARKING_AREA,LAND_VALUE,LAST_SALE_PRICE,LAST_SALE_DATE,MARKET_VALUE,LAST_SALE_UPSF
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1701 N 3RD ST,63301-2119,ALEXANDERS SUB ST CHAS,St Charles,40X150,2005.0,R - 30 - Average,3.0,2.0,0.0,5.0,2STY - 2 Story Building,"1,016",508,508,0,276,"$25,000.00","$118,750.00",01/16/2018,"$141,622.00",$116.88
2,901 S 5TH ST,63301,NaN,St Charles,0.3086 AC,2005.0,C - Not used,0.0,0.0,0.0,0.0,MEDICAL OFFICE,"2,994","2,994",0,0,0,"$110,905.00",$0.00,01/18/2005,"$561,853.00",$0.00
3,707 WATER ST,63301,NaN,St Charles,40X117,2005.0,R - 35 - Average to Good,4.0,3.0,1.0,7.0,15ST - 1 1/2 Story Building,"1,722","1,722",754,380,440,"$21,429.00","$305,300.00",03/11/2021,"$249,965.00",$177.29
4,100 N KINGSHIGHWAY,63301,OREARS SUB ST CHAS,St Charles,0.4400 AC,2005.0,C - Not used,0.0,0.0,0.0,0.0,SNACK BAR,"2,000","2,000",0,0,0,"$158,120.00","$1,100,000.00",04/07/2021,"$398,955.00",$550.00


In [78]:
# Average yearly mortgage rates

avg_mortgage_rate = {2000 : 7.72,
                     2001 : 6.5,
                     2002 : 5.98,
                     2003 : 5.17,
                     2004 : 5.21,
                     2005 : 5.42,
                     2006 : 6.07,
                     2007 : 6.03,
                     2008 : 5.62,
                     2009 : 4.57,
                     2010 : 4.1,
                     2011 : 3.7,
                     2012 : 2.93,
                     2013 : 3.11,
                     2014 : 3.29,
                     2015 : 3.09,
                     2016 : 2.93,
                     2017 : 3.28,
                     2018 : 4,
                     2019 : 3.39,
                     2020 : 2.61,
                     2021 : 2.25}


In [79]:
# Reading unseen dataset
unseen_data = file_path.replace("input-data\\st-charles\\","")
unseen_data = pd.read_csv(unseen_data + "unseen_data.csv")
unseen_data['BASEMENT_FINISHED_AREA'] = 790 # This is applicable only to 559 Fortress Ct, St Charles

# Filtering source dataset as per unseen data

subdivision = unseen_data['SUBDIVISION'].values
address = unseen_data['MAILING_ADDR'].values
living_area = unseen_data['BASEMENT_FINISHED_AREA'] + unseen_data['TOTAL_AREA'].str.replace('[,$]','', regex=True).astype('float64')
unseen_data_backup = unseen_data.copy()

df_new = df_new[df_new['SUBDIVISION'].isin(subdivision)].copy()
df_new = df_new[~df_new['MAILING_ADDR'].isin(address)].copy().reset_index(drop=True)


In [80]:
df_new = df_backup.copy()

# Removing any rows without sale year

df_new = df_new[df_new.LAST_SALE_DATE.notnull()].copy().reset_index(drop=True)


# Fixing the lot size
df_new['start'] = np.where(df_new.LOT_SIZE.str.find('AC') == -1, df_new.LOT_SIZE.str.split('X') , df_new.LOT_SIZE.str.replace(pat=" AC", repl=""))
df_new['start'][df_new.LOT_SIZE.str.find('AC') == -1] = df_new['start'][df_new.LOT_SIZE.str.find('AC') == -1].apply(lambda x: list(map(float, x)))
df_new['start'][df_new.LOT_SIZE.str.find('AC') == -1]  = df_new['start'][df_new.LOT_SIZE.str.find('AC') == -1].apply(lambda x: np.prod(x[0:2])/43560)
df_new['LOTS_SIZE'] = pd.to_numeric(df_new['start'])

# Fixing all numeric data types
df_new['TOTAL_AREA'] = df_new['TOTAL_AREA'].str.replace('[,$]','', regex=True).astype('float64')
df_new['BASE_AREA'] = df_new['BASE_AREA'].str.replace('[,$]','', regex=True).astype('float64')
df_new['BASEMENT_AREA'] = df_new['BASEMENT_AREA'].str.replace('[,$]','', regex=True).astype('float64')
df_new['BASEMENT_FINISHED_AREA'] = df_new['BASEMENT_FINISHED_AREA'].str.replace('[,$]','', regex=True).astype('float64')
df_new['LIVING_AREA'] = df_new['BASEMENT_FINISHED_AREA'] + df_new['TOTAL_AREA']
df_new['FINISHED_BASEMENT'] = np.where(df_new['BASEMENT_FINISHED_AREA'] == 0, 0, 1)
df_new['PARKING_AREA'] = df_new['PARKING_AREA'].str.replace('[,$]','', regex=True).astype('float64')
df_new['LAND_VALUE'] = df_new['LAND_VALUE'].str.replace('[,$]','', regex=True).astype('float64')
df_new['LAST_SALE_PRICE'] = df_new['LAST_SALE_PRICE'].str.replace('[,$]','', regex=True).astype('float64')
df_new = df_new[df_new['LAST_SALE_PRICE'] != 0].copy().reset_index(drop=True)
df_new['MARKET_VALUE'] = df_new['MARKET_VALUE'].str.replace('[,$]','', regex=True).astype('float64')
df_new['LAST_SALE_UPSF'] = df_new['LAST_SALE_UPSF'].str.replace('[,$]','', regex=True).astype('float64')
df_new['NEW_SALE_UPSF'] = df_new['LAST_SALE_PRICE'] / df_new['LIVING_AREA']

# Extracting features from date values
df_new['LAST_SALE_DATE'] = pd.to_datetime(df_new['LAST_SALE_DATE'])

# Generating useful information from date values

df_new['SALE_DATE_YR'] = df_new['LAST_SALE_DATE'].dt.year
df_new['AGE_OF_HOUSE_AT_SALE'] = df_new['SALE_DATE_YR'] - df_new['YEAR_BUILT']
df_new['SALE_DATE_MNTH'] = df_new['LAST_SALE_DATE'].dt.month
df_new['SALE_DATE_DT'] = df_new['LAST_SALE_DATE'].dt.day
df_new['MORTGAGE_RATE'] = df_new['SALE_DATE_YR'].map(avg_mortgage_rate)


C:\Users\sivac\miniconda3\envs\myDefaultEnv\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Users\sivac\miniconda3\envs\myDefaultEnv\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [81]:
# Removed Items - 'QUALITY_CODE', 'ARCHITECTURAL_STYLE', 'SALE_DATE_MNTH', 'LOTS_SIZE', 'SALE_DATE_YR', 'BASEMENT_AREA',

final = df_new[[ 'BEDROOMS', 'BATHROOMS','HALF_BATHROOMS', 'TOTAL_ROOMS', 'TOTAL_AREA',
          'BASE_AREA',  'BASEMENT_FINISHED_AREA', 'PARKING_AREA','LAND_VALUE', 'MARKET_VALUE',
           'AGE_OF_HOUSE_AT_SALE', 'LAST_SALE_UPSF', 'MORTGAGE_RATE']].copy().drop_duplicates().reset_index(drop=True)

# final = df_new[[ 'BEDROOMS', 'BATHROOMS','HALF_BATHROOMS', 'TOTAL_ROOMS', 'LIVING_AREA',
#           'FINISHED_BASEMENT', 'PARKING_AREA','LAND_VALUE', 'MARKET_VALUE',
#            'AGE_OF_HOUSE_AT_SALE', 'NEW_SALE_UPSF', 'MORTGAGE_RATE']].copy().drop_duplicates().reset_index(drop=True)

final = pd.get_dummies(final)
final.head()

,BEDROOMS,BATHROOMS,HALF_BATHROOMS,TOTAL_ROOMS,LIVING_AREA,FINISHED_BASEMENT,PARKING_AREA,LAND_VALUE,MARKET_VALUE,AGE_OF_HOUSE_AT_SALE,NEW_SALE_UPSF,MORTGAGE_RATE
0,4.0,2.0,2.0,0.0,2841.0,1,400.0,45000.0,273957.0,0.0,82.376980,5.42
1,3.0,2.0,1.0,7.0,1660.0,0,400.0,0.0,228286.0,0.0,130.055422,5.42
2,4.0,2.0,1.0,8.0,2612.0,1,400.0,45000.0,271709.0,14.0,93.223583,3.39
3,4.0,2.0,1.0,8.0,2923.0,1,400.0,45000.0,271535.0,0.0,75.325351,5.42
4,4.0,2.0,1.0,7.0,2088.0,0,440.0,45000.0,266154.0,11.0,109.674330,2.93


# Data Transformation

In [82]:
# df_new['LAST_SALE_DATE'] = pd.to_datetime(df_new['LAST_SALE_DATE'], format="%b %d %Y %I:%M:%S:%f%p")

# # Generating useful information from date values

# df_new['SALE_DATE_YR'] = df_new['LAST_SALE_DATE'].dt.year
# df_new['AGE_OF_HOUSE_AT_SALE'] = df_new['SALE_DATE_YR'] - df_new['YRBLT']
# df_new['SALE_DATE_MNTH'] = df_new['LAST_SALE_DATE'].dt.month
# df_new['SALE_DATE_DT'] = df_new['LAST_SALE_DATE'].dt.day
# df_new.drop(['LAST_SALE_DATE', 'YRBLT', 'USPF', 'PHYS'], axis=1, inplace=True)
# df_new = pd.get_dummies(df_new)
# df_new.head()

In [85]:
from sklearn.model_selection import train_test_split

# df_y = final['LAST_SALE_UPSF']
# df_x = final.drop('LAST_SALE_UPSF', axis=1)

df_y = final['NEW_SALE_UPSF']
df_x = final.drop('NEW_SALE_UPSF', axis=1)


x_train, x_test, y_train, y_test = train_test_split(df_x, df_y, test_size = 0.25, random_state = 2)
print(f' Training dataset shape: {x_train.shape} | Testing dataset shape: {x_test.shape}')

 Training dataset shape: (101, 11) | Testing dataset shape: (34, 11)


In [86]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
# from sklearn.model_selection import RandomForest
from sklearn import metrics

model = LinearRegression()
model = Lasso()
model.fit(x_train, y_train)

# print(model.intercept_)
# print(model.coef_)
y_pred = model.predict(x_test)

df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
print(f'{df.head()} \n')

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print('R-squared:', metrics.r2_score(y_test, y_pred, multioutput='uniform_average'))

# https://www.analyticsvidhya.com/blog/2021/05/all-you-need-to-know-about-your-first-machine-learning-model-linear-regression/
# https://www.analyticsvidhya.com/blog/2021/07/an-introduction-to-linear-regression/
# https://www.analyticsvidhya.com/blog/2021/06/getting-started-with-machine-learning%e2%80%8a-%e2%80%8aimplementing-linear-regression-from-scratch/


         Actual   Predicted
48   110.709763   90.285830
54   104.472304   97.211227
111  120.366332  120.942023
5    147.251309  147.768029
108   90.254237  110.336016 

Mean Absolute Error: 20.36221203512187
Mean Squared Error: 1249.678113779678
Root Mean Squared Error: 35.35078660765101
R-squared: -0.19059572068414043


# Predicting unknown dataset

In [88]:
unseen_data = unseen_data_backup.copy()

# Fixing the lot size
unseen_data['start'] = np.where(unseen_data.LOT_SIZE.str.find('AC') == -1, unseen_data.LOT_SIZE.str.split('X') , unseen_data.LOT_SIZE.str.replace(pat=" AC", repl=""))
unseen_data['start'][unseen_data.LOT_SIZE.str.find('AC') == -1] = unseen_data['start'][unseen_data.LOT_SIZE.str.find('AC') == -1].apply(lambda x: list(map(float, x)))
unseen_data['start'][unseen_data.LOT_SIZE.str.find('AC') == -1]  = unseen_data['start'][unseen_data.LOT_SIZE.str.find('AC') == -1].apply(lambda x: np.prod(x[0:2])/43560)
unseen_data['LOTS_SIZE'] = pd.to_numeric(unseen_data['start'])

# Fixing all numeric data types
unseen_data['TOTAL_AREA'] = unseen_data['TOTAL_AREA'].str.replace('[,$]','', regex=True).astype('float64')
unseen_data['BASE_AREA'] = unseen_data['BASE_AREA'].str.replace('[,$]','', regex=True).astype('float64')
# unseen_data['BASEMENT_AREA'] = unseen_data['BASEMENT_AREA'].str.replace('[,$]','', regex=True).astype('float64')
# unseen_data['BASEMENT_FINISHED_AREA'] = unseen_data['BASEMENT_FINISHED_AREA'].str.replace('[,$]','', regex=True).astype('float64')
# unseen_data['PARKING_AREA'] = unseen_data['PARKING_AREA'].str.replace('[,$]','', regex=True).astype('float64')
unseen_data['LIVING_AREA'] = unseen_data['TOTAL_AREA'] + unseen_data['BASEMENT_FINISHED_AREA'] 
unseen_data['FINISHED_BASEMENT'] = np.where(unseen_data['BASEMENT_FINISHED_AREA'] == 0, 0, 1)
unseen_data['LAND_VALUE'] = unseen_data['LAND_VALUE'].str.replace('[,$]','', regex=True).astype('float64')
# unseen_data['LAST_SALE_PRICE'] = unseen_data['LAST_SALE_PRICE'].str.replace('[,$]','', regex=True).astype('float64')
# unseen_data = unseen_data[unseen_data['LAST_SALE_PRICE'] != 0].copy().reset_index(drop=True)
unseen_data['MARKET_VALUE'] = unseen_data['MARKET_VALUE'].str.replace('[,$]','', regex=True).astype('float64')
# unseen_data['LAST_SALE_UPSF'] = unseen_data['LAST_SALE_UPSF'].str.replace('[,$]','', regex=True).astype('float64')


# Extracting features from date values
unseen_data['LAST_SALE_DATE'] = pd.to_datetime(unseen_data['LAST_SALE_DATE'])

# Generating useful information from date values

unseen_data['SALE_DATE_YR'] = dt.date.today().year
unseen_data['AGE_OF_HOUSE_AT_SALE'] = (dt.date.today().year)  - (unseen_data['YEAR_BUILT'])
unseen_data['SALE_DATE_MNTH'] = dt.date.today().month
unseen_data['SALE_DATE_DT'] = dt.date.today().day
unseen_data['MORTGAGE_RATE'] = unseen_data['SALE_DATE_YR'].map(avg_mortgage_rate)


unseen_data = unseen_data[[ 'BEDROOMS', 'BATHROOMS','HALF_BATHROOMS', 'TOTAL_ROOMS', 'TOTAL_AREA',
          'BASE_AREA',  'BASEMENT_FINISHED_AREA', 'PARKING_AREA','LAND_VALUE', 'MARKET_VALUE',
           'AGE_OF_HOUSE_AT_SALE', 'MORTGAGE_RATE']].copy().drop_duplicates().reset_index(drop=True)

# unseen_data = unseen_data[[ 'BEDROOMS', 'BATHROOMS','HALF_BATHROOMS', 'TOTAL_ROOMS', 'LIVING_AREA',
#           'FINISHED_BASEMENT', 'PARKING_AREA','LAND_VALUE', 'MARKET_VALUE',
#            'AGE_OF_HOUSE_AT_SALE', 'MORTGAGE_RATE']].copy().drop_duplicates().reset_index(drop=True)

unseen_data = pd.get_dummies(unseen_data)


C:\Users\sivac\miniconda3\envs\myDefaultEnv\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\sivac\miniconda3\envs\myDefaultEnv\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [89]:
predicted_price = model.predict(unseen_data)
df = pd.DataFrame({'Address': address, 'Predicted UPSF': predicted_price, 'Living Area': living_area})
df['Predicted Sale Price'] = df['Predicted UPSF'] * df['Living Area']
df

,Address,Predicted UPSF,Living Area,Predicted Sale Price
0,559 FORTRESS CT,107.232143,2978.0,319337.320613


In [53]:
unseen_data_backup

,ACCOUNT,SITUS,CITY,ZIP,OWNER,MAILING_ADDR,MAILING_CITY,MAILING_STATE,MAILING_ZIP,SUBDIVISION,SCHOOL_DISTRICT,NEIGHBORHOOD_CODE,LOT_SIZE,YEAR_BUILT,QUALITY_CODE,BEDROOMS,BATHROOMS,HALF_BATHROOMS,TOTAL_ROOMS,ARCHITECTURAL_STYLE,TOTAL_AREA,BASE_AREA,PARKING_AREA,BASEMENT_AREA,BASEMENT_FINISHED_AREA,MARKET_VALUE,LAND_VALUE,LAST_SALE_PRICE,LAST_ADJUSTED_SALE_PRICE,LAST_SALE_DATE,LAST_SALE_QUALIFIED,LAST_SALE_UPSF
0,T040800021,559 FORTRESS CT,St. Charles,63303,MOSQUERA MARIA LUISA LERMA,559 FORTRESS CT,ST CHARLES,MO,63303,ENCLAVE AT MONTCLAIR,Francis Howell,1345,0.1000 AC,2004,R - 35 - Average to Good,4,2,1,7,2STY - 2 Story Building,"2,188","1,284",400,904,0,"$274,831.00","$45,000.00","$258,000.00","$258,000.00",7/15/2016,Y,$117.92


# Visualizations

In [ ]:
numericCols = df_new.select_dtypes(include=['float64','int64'])

# Calculate Correlation
corr = numericCols.corr()
# Generate a mask for the upper triangle
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})